In [1]:
import wget
import ssl
import os
ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
# Enter NCRB Individual PDF url
pdf_url = 'https://ncrb.gov.in/sites/default/files/adsi_reports_previous_year/Table-1A.6_2019.pdf'
fileName = pdf_url[67:]
try:
    #Delete the file to redownload it
    os.remove(fileName)
except FileNotFoundError as e:
    #Ignore in case FileNotFound Error occurs
    pass
# Download report
fileName = wget.download(pdf_url,fileName)

In [3]:
import numpy as np
import pandas as pd
import camelot

In [4]:
fileName

'Table-1A.6_2019.pdf'

In [5]:
tables = camelot.read_pdf(fileName,
                          flavor='stream',
                          table_areas=[
                              "41.84432956381261,708.3528537908206,577.6441357027464,55"
                              ],
                          columns=[
                              "74.06927302100162,184.69221324717287,227.97945072697902,273.1905654281099,311.6681098546042,352.06953150242333,395.3567689822294,436.7201292407109,482.8931825525041,520"
                              ],
                          split_text=True,
                          pages="1-6"
                         )

In [6]:
cityInd = [1,3]

In [7]:
for each in cityInd:
    print(tables[each].df.head())

    0          1        2        3    4    5    6    7    8     9     10
0                  (Night)  (Night)                                     
1  (1)        (2)      (3)      (4)  (5)  (6)  (7)  (8)  (9)  (10)  (11)
2                       15       28   43   33   40   40   47    46   292
3   37                AGRA                                              
4   38  AHMEDABAD       84       48  124  237  224  208  261   194  1380
    0          1     2     3     4     5     6     7     8     9     10
0  (1)        (2)  (12)  (13)  (14)  (15)  (16)  (17)  (18)  (19)  (20)
1                     1     4     7    10     6     6     5     3    42
2   37             AGRA                                                
3   38  AHMEDABAD     0     0     0     0     0     0     0     0     0
4   39  ALLAHABAD     0     0     0     0     0     0     0     0     0


In [8]:
#Fixes Wierd Error casue by Camelot
# At Page 2
tables[1].df.iloc[2,1] = 'AGRA'
tables[1].df.iloc[2,0] = tables[1].df.iloc[3,0]
tables[1].df.drop(index=3, inplace=True)
tables[1].df.index = range(len(tables[1].df))
# At Page 4
tables[3].df.iloc[1,1] = 'AGRA'
tables[3].df.iloc[1,0] = tables[1].df.iloc[2,0]
tables[3].df.drop(index=2, inplace=True)
tables[3].df.index = range(len(tables[3].df))

In [9]:
for each in cityInd:
    print(tables[each].df.head())

    0          1        2        3    4    5    6    7    8     9     10
0                  (Night)  (Night)                                     
1  (1)        (2)      (3)      (4)  (5)  (6)  (7)  (8)  (9)  (10)  (11)
2   37       AGRA       15       28   43   33   40   40   47    46   292
3   38  AHMEDABAD       84       48  124  237  224  208  261   194  1380
4   39  ALLAHABAD       87       83  122   86   93   99  123    83   776
    0          1     2     3     4     5     6     7     8     9     10
0  (1)        (2)  (12)  (13)  (14)  (15)  (16)  (17)  (18)  (19)  (20)
1   37       AGRA     1     4     7    10     6     6     5     3    42
2   38  AHMEDABAD     0     0     0     0     0     0     0     0     0
3   39  ALLAHABAD     0     0     0     0     0     0     0     0     0
4   40   AMRITSAR     0     0     0     0     0     0     0     0     0


In [10]:
tables[1].df.loc[36:39]

,0,1,2,3,4,5,6,7,8,9,10
36,71,MEERUT,41,33,42,51,65,86,98,95,511
37,,,57,65,71,77,84,77,71,86,588
38,,MUMBAI,72,,,,,,,,
39,73,NAGPUR,55,54,108,187,171,188,189,167,1119


In [11]:
# At Page 2
tables[1].df.loc[37,1] = 'MUMBAI'
tables[1].df.loc[37,0] = tables[1].df.loc[38,2]
tables[1].df.drop(index=38, inplace=True)
tables[1].df.index = range(len(tables[1].df))

In [12]:
tables

<TableList n=6>

In [13]:
def printnAllTables(tables):
    for i in range(len(tables)):
        print("Page No: ",(i+1))
        print(tables[i].df)
        print()

In [14]:
# Incase we're intersted in seeing the original scraped tables.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

In [15]:
# Removes Garbage Rows that may have been detected Before the 'STATES' row
# AND
# Removes Garbage Rows that may have been detected after the 'TOTAL (ALL INDIA)' row
def clean(table):
    while(table.df.iloc[0,1] != 'STATES' and table.df.iloc[0,1] != 'AGRA'):
        table.df = table.df.iloc[1:]
    while(table.df.iloc[-1,1] != 'TOTAL (ALL INDIA)' and table.df.iloc[-1,1] != 'TOTAL (CITIES)'):
        table.df = table.df.iloc[:-1]
    # The Return line is unnecessary due to all changes being reflected in the original references
    return table

In [16]:
# Splits the second column into 1st column index and 2nd column test when faulty/required
def split_by_first_space(df, strIndex, stpIndex):
    df = df.iloc[strIndex:stpIndex+1]
    for index, each in df.iterrows():
        space = each[1].find(' ')
        each[0] = each[1][:space]
        each[1] = each[1][space:]

In [17]:
def mergeRows(df, strIndex, stpIndex):
    DF = df.loc[strIndex:stpIndex]
    df = DF.T
    l = []
    for index, each in df.iterrows():
        output = ''
        for ind, cell in each.iteritems():
            output += (str(cell) + ' ')
#         if(output == ''):
#             continue
        l.append(output.strip())
    l = pd.Series(l,name=str(strIndex))
    DF.loc[strIndex] = l

In [18]:
for each_table in tables:
    clean(each_table)

In [19]:
# Incase we're intersted in seeing the scraped tables at this point.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

In [20]:
table7_8 = camelot.read_pdf(fileName,
                          flavor='stream',
                          table_areas=[
                              "41.84432956381261,708.3528537908206,577.6441357027464,55"
                              ],
                          columns=[
                              "74.06927302100162,184.69221324717287,227.97945072697902,273.1905654281099,311.6681098546042,352.06953150242333,395.3567689822294,436.7201292407109,482.8931825525041,510"
                              ],
                          split_text=True,
                          pages="7-8"
                         )

In [21]:
for each_table in table7_8:
    clean(each_table)

In [22]:
tableList = [each.df.copy() for each in tables] + [each.df.copy() for each in table7_8]
states = [ each.copy() for index,each in enumerate(tableList) if(index%2==0)]
cities = [ each.copy() for index,each in enumerate(tableList) if(index%2==1)]

In [23]:
# List of Column headers as column are nested, i.e exhibit a multi level/index structure
# The multi level structre will be generated and applied in the next 
time_header = [
    '0000 hrs to 0300 hrs. (Night)',
    '0300 hrs to 0600 hrs. (Night)',
    '0600 hrs to 0900 hrs (Day)',
    '0900 hrs to 1200 hrs (Day)',
    '1200 hrs to 1500 hrs (Day)',
    '1500 hrs to 1800 hrs (Day)',
    '1800 hrs to 2100hrs (Night)',
    '2100 hrs. to 2400hrs(Night)',
    'Total'
]
column_header = [
    'Road Accidents',
    'Railway Crossing Accidents',
    'Railway Accidents',
    'Total(Traffic Accidents)'
]
len(time_header)

9

In [24]:
def set_Column(df, title, state):
    column_list = [
        ("Sl. No.",''),
        ("State/UT" if(state) else "City",''), 
        (title,time_header[0]),
        (title,time_header[1]),
        (title,time_header[2]),
        (title,time_header[3]),
        (title,time_header[4]),
        (title,time_header[5]),
        (title,time_header[6]),
        (title,time_header[7]),
        (title,time_header[8])
    ]
    df.columns = pd.MultiIndex.from_tuples(column_list)

In [25]:
for i,(state,city) in enumerate(zip(states,cities)):
    set_Column(df=city, title=column_header[i], state=False)
    city.set_index(["Sl. No.","City"],inplace=True)
    set_Column(df=state, title=column_header[i], state=True)
    state.set_index(["Sl. No.","State/UT"],inplace=True)

In [26]:
for state in states:
    state.index = states[0].index.copy()

In [27]:
State = pd.concat(states,axis=1)
City = pd.concat(cities,axis=1)

In [28]:
# Incase we're intersted in seeing the scraped tables at this point.
# Commented as it takes too much screen space
# Run if you want
# printnAllTables(tables)

# OR Print a simgle one according to your choice
# tables[23].df

In [29]:
# Reset Index before saving file, for better formatting in RAW CSV
State.reset_index().to_csv("Time of Occurrence – wise Number of Traffic Accidents during 2019 (StateUT-wise).csv",index=False)
City.reset_index().to_csv("Time of Occurrence – wise Number of Traffic Accidents during 2019 (City-wise).csv",index=False)